In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ASUS\\Desktop\\ML-OPs\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\ASUS\\Desktop\\ML-OPs\\Kidney-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weight: str
    params_classes: int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weight = self.params.WEIGHTS,
            params_classes = self.params.CLASSES
        )

        return prepare_base_model_config 

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[ 2025-06-22 16:17:28,023 ] DEBUG - tpu_cluster_resolver - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
[ 2025-06-22 16:17:29,506 ] DEBUG - __init__ - Creating converter from 7 to 5
[ 2025-06-22 16:17:29,511 ] DEBUG - __init__ - Creating converter from 5 to 7
[ 2025-06-22 16:17:29,519 ] DEBUG - __init__ - Creating converter from 7 to 5
[ 2025-06-22 16:17:29,523 ] DEBUG - __init__ - Creating converter from 5 to 7
[ 2025-06-22 16:17:31,164 ] DEBUG - path - etils.epath was not found. Using pathlib for file I/O.
[ 2025-06-22 16:17:33,170 ] DEBUG - __init__ - pydot initializing
[ 2025-06-22 16:17:33,171 ] DEBUG - __init__ - pydot 4.0.1
[ 2025-06-22 16:17:33,177 ] DEBUG - core - pydot core module initializing


In [14]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weight,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    @staticmethod
    def _prepare_full_model(model: tf.keras.Model, classes: int, freeze_all: bool, freeze_till: int, learning_rate: float):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

In [15]:
from cnnClassifier import MyException
import sys
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise MyException(e, sys) from e

[ 2025-06-22 16:53:43,855 ] INFO - common - yaml file: config\config.yaml loaded successfully.
[ 2025-06-22 16:53:43,858 ] INFO - common - yaml file: params.yaml loaded successfully.
[ 2025-06-22 16:53:43,860 ] INFO - common - Created directory at: artifacts
[ 2025-06-22 16:53:43,861 ] INFO - common - Created directory at: artifacts/prepare_base_model
[ 2025-06-22 16:53:44,359 ] WARNING - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   